In [1]:
import os
import sys
import scanpy as sc
import tensorflow as tf
import numpy as np
import pandas as pd
from scaden import Scaden
from architectures import architectures
from scaden_main import *
import scanpy as sc
from tqdm import tqdm

G:\Users\Philipp\Anaconda3\lib\site-packages\scanpy\api\__init__.py:6: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
from tqdm import tqdm

In [3]:
model_dir="."
input_path='processed.h5ad'
train_datasets=''
tf.reset_default_graph()
sess = tf.Session() 
cdn256 = Scaden(sess=sess,
                model_dir=model_dir + "\m256",
                model_name='m256')
cdn256.hidden_units = architectures['m256'][0]
cdn256.do_rates = architectures['m256'][1]

cdn256.build_model(input_path=input_path, train_datasets=train_datasets, mode="train")





Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_classes(iterator)`.


Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dropout instead.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






In [37]:
opt = tf.train.AdamOptimizer(learning_rate=0.0001)
gradient_step = opt.compute_gradients(cdn256.logits, tf.trainable_variables())

In [49]:
grad = tf.compat.v1.gradients(cdn256.logits, tf.trainable_variables()) 

In [45]:
cdn256.sess.run(tf.global_variables_initializer())

In [50]:
gradient  = sess.run(grad, feed_dict={cdn256.x_data_ph: cdn256.x_data})

TypeError: Fetch argument None has invalid type <class 'NoneType'>

In [51]:

print(grad)

[<tf.Tensor 'gradients_9/scaden_model/dense1/MatMul_grad/MatMul_1:0' shape=(13590, 256) dtype=float32>, <tf.Tensor 'gradients_9/scaden_model/dense1/BiasAdd_grad/BiasAddGrad:0' shape=(256,) dtype=float32>, <tf.Tensor 'gradients_9/scaden_model/dense2/MatMul_grad/MatMul_1:0' shape=(256, 128) dtype=float32>, <tf.Tensor 'gradients_9/scaden_model/dense2/BiasAdd_grad/BiasAddGrad:0' shape=(128,) dtype=float32>, None, None, None, None, <tf.Tensor 'gradients_9/scaden_model/logits_layer/MatMul_grad/MatMul_1:0' shape=(128, 2) dtype=float32>, <tf.Tensor 'gradients_9/scaden_model/logits_layer/BiasAdd_grad/BiasAddGrad:0' shape=(2,) dtype=float32>, None, None, None, None]


In [ ]:
cdn256.sess.run(tf.global_variables_initializer())
cdn256.sess.run(tf.local_variables_initializer())
cdn256.saver = tf.train.Saver()
model = os.path.join(cdn256.model_dir, cdn256.model_name)
cdn256.writer = tf.summary.FileWriter(model, cdn256.sess.graph)
cdn256.eval_writer = tf.summary.FileWriter(os.path.join(cdn256.model_dir, "eval"), cdn256.sess.graph)

# Initialize datasets
cdn256.sess.run(tf.global_variables_initializer())

cdn256.load_weights(cdn256.model_dir)

In [4]:
pbar = tqdm(range(cdn256.num_steps))
for _ in pbar:
    _, loss, summary = cdn256.sess.run([cdn256.optimizer, cdn256.loss, cdn256.merged_summary_op])
    cdn256.writer.add_summary(summary, tf.compat.v1.train.global_step(cdn256.sess, cdn256.global_step))
    description = "Step: " + str(tf.compat.v1.train.global_step(cdn256.sess, cdn256.global_step)) + ", Loss: {:4.3f}".format(loss)
    pbar.set_description(desc=description)

Step: 1000, Loss: 0.002: 100%|██████████| 1000/1000 [00:18<00:00, 54.08it/s]


In [7]:
grad = tf.gradients(cdn256.logits,cdn256.x)
print(cdn256.x)

Tensor("IteratorGetNext:0", shape=(?, 13590), dtype=float32)


In [9]:
gradient = cdn256.sess.run(grad, feed_dict={cdn256.training_mode: False}[0])

KeyError: 0

In [12]:
gradient[0]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [13]:
gradient[0].sum(axis=0)

array([ 1.9154087e-09,  3.1950771e-09, -3.2409841e-09, ...,
       -5.4603327e-10,  2.6471851e-09,  6.3801414e-10], dtype=float32)

In [23]:
print(cdn256.training_mode)

Tensor("PlaceholderWithDefault:0", shape=(), dtype=bool)


In [57]:
        cdn256.saver.save(cdn256.sess, model, global_step=cdn256.global_step)
        # Save features and celltypes
        pd.DataFrame(cdn256.labels).to_csv(cdn256.model_dir + "/celltypes.txt", sep="\t")
        pd.DataFrame(cdn256.sig_genes).to_csv(cdn256.model_dir + "/genes.txt", sep="\t")

In [14]:
cdn256.dropout

AttributeError: 'Scaden' object has no attribute 'dropout'

In [9]:
x = tf.placeholder(tf.float32)

In [18]:
 y =  2*x*x

In [25]:
var_grad = tf.gradients(y, x)
print(var_grad)

[<tf.Tensor 'gradients_5/AddN:0' shape=<unknown> dtype=float32>]


In [46]:
var_grad_val = cdn256.sess.run(var_grad,feed_dict={x:a}) 

TypeError: unhashable type: 'numpy.ndarray'

In [33]:
print(var_grad_val)

[8.0]


In [47]:
a=tuple([1,2])

In [48]:
print(a)

(1, 2)


In [30]:

batch_1=np.random.normal(0,1, [2, 3])
batch_2=np.random.normal(0,1, [2, 3])
x = tf.placeholder(tf.float32, shape=(None, 3))

out = tf.layers.dense(x, 2, tf.tanh,
                              kernel_initializer=tf.random_normal_initializer(
                                  stddev=np.sqrt(1 / 100)))
predictions = tf.layers.dense(out, 1, tf.tanh,
                              kernel_initializer=tf.random_normal_initializer(
                                  stddev=np.sqrt(1 / 100)))

opt = tf.train.GradientDescentOptimizer(learning_rate=0.01)
gradient_step = opt.compute_gradients(predictions, tf.trainable_variables())

sess=tf.Session()
sess.run(tf.global_variables_initializer())
gradients_1 = sess.run(gradient_step, feed_dict={x: batch_1})
gradients_2 = sess.run(gradient_step, feed_dict={x: batch_2})

TypeError: Fetch argument None has invalid type <class 'NoneType'>

In [53]:
batch_1=np.random.normal(0,1, [2, 3])
batch_2=np.random.normal(0,1, [2, 3])
x = tf.placeholder(tf.float32, shape=(None, 3))

predictions = tf.layers.dense(x, 1, tf.tanh,
                              kernel_initializer=tf.random_normal_initializer(
                                  stddev=np.sqrt(1 / 100)))

opt = tf.train.GradientDescentOptimizer(learning_rate=0.01)
gradient_step = opt.compute_gradients(predictions, tf.trainable_variables())

In [54]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())
gradients_1 = sess.run(gradient_step, feed_dict={x: batch_1})
gradients_2 = sess.run(gradient_step, feed_dict={x: batch_2})

TypeError: Fetch argument None has invalid type <class 'NoneType'>